In [1]:
### PACKAGES
import csv
import numpy as np
import cv2
import os
import matplotlib
import matplotlib.pyplot as plt
import pickle
import time
from copy import deepcopy
from asift import *
from tensorflow import keras
from keras import layers
from find_obj import init_feature, filter_matches, explore_match
from scipy import ndimage 
from skimage import transform
from skimage.transform import warp, ProjectiveTransform
from sklearn.neighbors import NearestNeighbors
import sys
import tensorflow as tf
from netvlad_keras import *
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model

In [2]:
### CONSTANTS
ROOTPATH = './'
IMG_HEIGHT = 128
IMG_WIDTH = 128
NUM_CHANNELS = 3

### Import Training and Test Data

In [3]:
train_img_names = [] # list of train image names as strings
train_img_labels = [] # list of train images labels as tuples (latitude, longitude)
with open('COMP90086_2021_Project_train/train.csv') as train_csv:
    reader = csv.reader(train_csv, delimiter=',')
    next(reader) # skip header row
    for row in reader:
        name = row[0] # string
        label = (float(row[1]), float(row[2])) # tuple
        train_img_names.append(name)
        train_img_labels.append(label)

# train images stored as numpy array
train_size = (len(train_img_names), IMG_HEIGHT, IMG_WIDTH,NUM_CHANNELS )
train_images = np.zeros(train_size, dtype='uint8')
for i in range(len(train_img_names)):
    name = train_img_names[i]
    subpath = 'COMP90086_2021_Project_train/train/' + name + '.jpg'
    img = cv2.imread(os.path.join(ROOTPATH, subpath))
    img = cv2.resize(img,(IMG_WIDTH,IMG_HEIGHT))
    train_images[i] = img

# train labels stored as numpy array
train_labels = np.array(train_img_labels)

In [4]:
test_img_names = [] # list of test image names as strings
with open('COMP90086_2021_Project_test/imagenames.csv') as test_csv:
    reader = csv.reader(test_csv, delimiter=',')
    next(reader) # skip header row
    for row in reader:
        name = row[0] # string
        test_img_names.append(name)
        
# test images stored as numpy array
test_size = (len(test_img_names), IMG_HEIGHT, IMG_WIDTH, NUM_CHANNELS)
test_images = np.zeros(test_size, dtype='uint8')
for i in range(len(test_img_names)):
    name = test_img_names[i]
    subpath = 'COMP90086_2021_Project_test/test/' + name + '.jpg'
    img = cv2.imread(os.path.join(ROOTPATH, subpath))
    img = cv2.resize(img,(IMG_WIDTH,IMG_HEIGHT))
    test_images[i] = img

### Baseline CNN

In [5]:
baseline = keras.models.Sequential()

baseline.add(layers.Input((IMG_HEIGHT, IMG_WIDTH, NUM_CHANNELS)))
baseline.add(layers.Conv2D(32, (3, 3), activation='relu'))
baseline.add(layers.Conv2D(16, (3, 3), activation='relu'))
baseline.add(layers.Dense(16, activation='relu'))
baseline.add(layers.Dense(2))

op = keras.optimizers.Adam()
baseline.compile(loss='mae', optimizer=op)

baseline.summary()

h = baseline.fit(train_images, train_labels,verbose = 1,batch_size=1,validation_split = 0.1,epochs = 20)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 126, 126, 32)      896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 124, 124, 16)      4624      
_________________________________________________________________
dense (Dense)                (None, 124, 124, 16)      272       
_________________________________________________________________
dense_1 (Dense)              (None, 124, 124, 2)       34        
Total params: 5,826
Trainable params: 5,826
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
6750/6750 [==============================] - 18s 2ms/step - loss: 40.9270 - val_loss: 39.1598
Epoch 2/20
6750/6750 [==============================] - 16s 2ms/step - loss: 40.3211 - val_loss: 38.3868
Epoch 3/20
6750/6750 [====================

## EXPERIMENTS FOR PAPER

### FEATURE EXTRACTION: SIFT

In [6]:
sift = cv2.SIFT_create() # initialise SIFT detector
sift_train_features = []
for i in range(len(train_images)):    
    train_image = train_images[i]
    # convert test image to grayscale
    gray_train_image = cv2.cvtColor(train_image, cv2.COLOR_BGR2GRAY)
    # SIFT keypoints and descriptors for test image
    _, desc = sift.detectAndCompute(gray_train_image, None)
    sift_train_features.append(desc)
    
with open(os.path.join(os.getcwd(),"feature_extraction/sift_train_feat.pickle"),'wb') as handle:
    pickle.dump(sift_train_features, handle, protocol=pickle.HIGHEST_PROTOCOL)


### FEATURE EXTRACTION: ASIFT

In [7]:
# Initialization
detector, matcher = init_feature("sift-flann")
pool=ThreadPool(processes = cv.getNumberOfCPUs())

asift_desc_features = [] # Save feature descriptions
asift_kp_feaures = [] # Save description key points


for i in range(len(train_images)):    
    train_image = train_images[i]
    gray_test_image = cv2.cvtColor(train_image, cv2.COLOR_BGR2GRAY)
    
    # ASIFT keypoints and descriptors for test image
    kp1, desc1 = affine_detect(detector, gray_test_image, pool=pool)
    
    # Cant Pickle (Save) cv2.keypoint directly, so we need to first serialize it 
    serialize_list = []
    for kp in kp1:
        temp = (kp.pt, kp.size, kp.angle, kp.response, kp.octave, kp.class_id) 
        serialize_list.append(temp)

    asift_desc_features.append(desc1)
    asift_kp_feaures.append(serialize_list)

# Pickle to save to disk 
with open(os.path.join(os.getcwd(),"feature_extraction/asift_train_feat.pickle"),'wb') as handle:
    pickle.dump(asift_desc_features, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(os.path.join(os.getcwd(),"feature_extraction/asift_kp_train_feat.pickle"),'wb') as handle:
    pickle.dump(asift_kp_feaures, handle, protocol=pickle.HIGHEST_PROTOCOL)

### FEATURE EXTRACTION: NetVLAD

In [8]:
netvlad_model = NetVLADModel(input_shape=(IMG_HEIGHT, IMG_WIDTH, NUM_CHANNELS)) # Define input layer

netvlad_model.summary()
netvlad_model.load_weights('checkpoint/netvlad_weights.h5')
netvlad_model.build()

# Extract features
start= time.time()
train_features = netvlad_model.predict(train_images)
end= time.time()
with open(os.path.join(os.getcwd(),"feature_extraction/vlad_train_feat.pickle"),'wb') as handle:
    pickle.dump(train_features, handle, protocol=pickle.HIGHEST_PROTOCOL)

print("Time Taken: ",end-start)

C:\Users\kazuy\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
substract_average (Substract (None, 128, 128, 3)       3         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 128, 128, 64)      1792      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 128, 128, 64)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 64, 64)        0         
_________________________________________________________________
activation (Activation)      (None, 64, 64, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 64, 64, 128)       73856     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 64, 64, 128)      

### FEATURE EXTRACTION: Self-Supervised Learning

In [9]:
# train images stored as numpy array
train_size = (len(train_img_names), 128, 128 ,NUM_CHANNELS )
train_images_ss = np.zeros(train_size, dtype='uint8')
for i in range(len(train_img_names)):
    name = train_img_names[i]
    subpath = 'COMP90086_2021_Project_train/train/' + name + '.jpg'
    img = cv2.imread(os.path.join(ROOTPATH, subpath))
    img = cv2.resize(img,(128,128))
    train_images_ss[i] = img
train_images_ss = train_images_ss.astype(float)/255
# train labels stored as numpy array
train_labels_ss = np.array(train_img_labels)


# test images stored as numpy array
test_size = (len(test_img_names), 128, 128, NUM_CHANNELS)
test_images_ss = np.zeros(test_size, dtype='uint8')
for i in range(len(test_img_names)):
    name = test_img_names[i]
    subpath = 'COMP90086_2021_Project_test/test/' + name + '.jpg'
    img = cv2.imread(os.path.join(ROOTPATH, subpath))
    img = cv2.resize(img,(128,128))
    test_images_ss[i] = img
test_images_ss = test_images_ss.astype(float)/255

In [10]:
# feature_extraction_method: choose from "SS ROT" or "SS WARP"
feature_extraction_method = "SS ROT"

# shuffle training instances
idx = np.random.randint(train_images_ss.shape[0], size=train_images_ss.shape[0])
train_images_ss_shuffled = train_images[idx,:,:,:]
train_labels_ss_shuffled = train_labels[idx,:]


In [11]:

if feature_extraction_method == "SS WARP": # Self Supervised With Warping
    train_images_warp=np.zeros_like(train_images_ss_shuffled)  
    train_label_warp=np.zeros((train_labels_ss_shuffled.shape[0],1) ) 
elif feature_extraction_method == "SS ROT": # Self Supervised With Warping
    train_images_rot=np.zeros_like(train_images_ss_shuffled)  
    train_label_rot=np.zeros((train_labels_ss_shuffled.shape[0],1) )
    
#0: no warping, 1 warp right, 2 warp left, 3 warp top, 4 warp bottom
for i in range (train_images_ss_shuffled.shape[0]):
    
    img= train_images_ss_shuffled[i,:,:,:] 
    
    if feature_extraction_method == "SS ROT":
        
        if np.mod(i,4)==0:
            rot_img = img 
            rot_lab=0
        elif np.mod(i,4)==1:
            rot_img = ndimage.rotate(img, 90)
            rot_lab=1
        elif np.mod(i,4)==2:
            rot_img = ndimage.rotate(img, 180)
            rot_lab=2
        else:
            rot_img = ndimage.rotate(img, 270)
            rot_lab=3
            
        train_images_rot[i,:,:,:] = rot_img
        train_label_rot[i,0]=rot_lab
    
    elif feature_extraction_method == "SS WARP":
        
        if np.mod(i,5)==0:
            warped_img = img 
            warped_label = 0
        elif np.mod(i,5)==1:
            warped_img = warp(img, transform_right)
            warped_label=1
        elif np.mod(i,5)==2:
            warped_img = warp(img, transform_left)
            warped_label=2
        elif np.mod(i,5)==3:
            warped_img = warp(img, transform_top)
            warped_label=3
        else:
            warped_img = warp(img, transform_bottom)
            warped_label=4

        train_images_warp[i,:,:,:] = warped_img
        train_label_warp[i,0]=warped_label



In [12]:
ss_model = keras.Sequential(
    [
        layers.Input((IMG_HEIGHT, IMG_WIDTH, 3)),
        layers.Conv2D(16, (3, 3), activation='relu'), # 126
        layers.Conv2D(16, (3, 3), activation='relu'), # 124
        layers.MaxPooling2D((3, 3),padding='same'), # 42
   
        layers.Conv2D(32, (3, 3), activation='relu'), # 40
        layers.Conv2D(32, (3, 3), activation='relu'), # 38
        layers.MaxPooling2D((3, 3),padding='same'), # 13
 
        layers.Conv2D(64, (3, 3), activation='relu'), # 11
        layers.Conv2D(64, (3, 3), activation='relu'), # 9
        layers.MaxPooling2D((3, 3),padding='same'), # 3
        layers.MaxPooling2D((2, 2),padding='same'), # 2
        
        layers.Flatten(),
        layers.Dense(256, activation='relu'),
        layers.Dense(5, activation='softmax')
        #layers.Dense(4, activation='softmax')
    ]
)

ss_model.summary()

#compile model
loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False)

ss_model.compile(optimizer='adam', loss=[loss], metrics=['accuracy'])
history=ss_model.fit(train_images_rot,train_label_rot, verbose = 1,validation_split = 0.2,
                           epochs=10, batch_size=100)


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 126, 126, 16)      448       
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 124, 124, 16)      2320      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 42, 42, 16)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 40, 40, 32)        4640      
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 38, 38, 32)        9248      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 11, 11, 64)       

In [13]:
# Use partial model for feature extraction
features_model=Model(ss_model.inputs, ss_model.layers[-3].output) 

train_features = features_model.predict(train_images_ss) # Extract Features

file_name = feature_extraction_method.replace(" ", "_").lower()

with open(os.path.join(os.getcwd(),"feature_extraction/{}_train_feat.pickle".format(file_name)),'wb') as handle:
    pickle.dump(train_features, handle, protocol=pickle.HIGHEST_PROTOCOL)

### FEATURE MATCHING: SELECT METHOD

In [16]:
# feature_config: can choose from "SIFT","ASIFT","SS ROT", "SS WARP", "VLAD"
feature_config = "SIFT"

if feature_config == "SIFT": 
    if os.path.exists('./feature_extraction/sift_train_feat.pickle'):
        with open(os.path.join("feature_extraction","sift_train_feat.pickle"),'rb') as handle:
            img_features = pickle.load(handle)
        print("Using Features",feature_config)
        
elif feature_config == "ASIFT":
    if os.path.exists('./feature_extraction/asift_train_feat.pickle') and os.path.exists('./feature_extraction/asift_kp_train_feat.pickle'):
        with open(os.path.join("feature_extraction","asift_train_feat.pickle"),'rb') as handle:
            img_features = pickle.load(handle)
        with open(os.path.join("feature_extraction","asift_kp_train_feat.pickle"),'rb') as handle:
            serialized_list = pickle.load(handle)
            
        # Need to deserialize back into cv2.Keypoint object (we couldnt save Keypoint objects directly using Pickle)
        img_kp = []
        for img in serialized_list:
            tmp_kp_list = []
            for kp in img:
                temp = cv2.KeyPoint(x=kp[0][0],y=kp[0][1], size=kp[1], angle=kp[2], response=kp[3], octave=kp[4], class_id=kp[5]) 
                tmp_kp_list.append(temp)
            img_kp.append(tmp_kp_list)            
        print("Using Features",feature_config)

elif feature_config == "VLAD": # NetVLAD 
    if os.path.exists('./feature_extraction/VLAD_train_feat.pickle'):
        with open(os.path.join("feature_extraction","VLAD_train_feat.pickle"),'rb') as handle:
            img_features = pickle.load(handle)
        print("Using Features",feature_config)

elif feature_config == "SS ROT": # Self-supervised (Rotation)
    if os.path.exists('./feature_extraction/ss_rot_train_feat.pickle'):
        with open(os.path.join("feature_extraction","ss_rot_train_feat.pickle"),'rb') as handle:
            img_features = pickle.load(handle)

        print("Using Features",feature_config)

elif feature_config == "SS WARP": #Self-supervised (Warp)
    if os.path.exists('./feature_extraction/ss_warp_train_feat.pickle'):
        with open(os.path.join("feature_extraction","ss_warp_train_feat.pickle"),'rb') as handle:
            img_features = pickle.load(handle)

        print("Using Features",feature_config)



Using Features SIFT


### FEATURE MATCHING: FLANN

In [18]:
if feature_config == "VLAD" or feature_config == "SS WARP" or feature_config == "SS ROT":
    print("Cannt use FLANN with {}, please use a different descriptor".format(feature_config))
    assert(False)
    

print("USING FEATURES",feature_config)

# Shuffle Features
idx = np.random.randint(len(img_features), size=len(img_features)) # Scramble index values
img_shuffled = [img_features[i] for i in idx] # Shuffle descriptors
labels_shuffled = [train_labels[i] for i in idx] # Shuffle image labels

if feature_config == "ASIFT": # We need the keypoints of the description for ASIFT but not for SIFT
    kp_shuffled = [img_kp[i] for i in idx] # Shuffle keypoints
    train_kp_shuffled = kp_shuffled[:int(len(img_shuffled)*.80)] 
    vali_kp_shuffled = kp_shuffled[int(len(img_shuffled)*.80):]
    
# Train set (80%)
train_features_shuffled = img_shuffled[:int(len(img_shuffled)*.80)]
train_labels_shuffled = labels_shuffled[:int(len(labels_shuffled)*.80)]

# Validation set (20%)
vali_features_shuffled = img_shuffled[int(len(img_shuffled)*.80):]
vali_labels_shuffled = labels_shuffled[int(len(labels_shuffled)*.80):]

# FLANN parameters
k_nn = 2
lowe_ratio = 0.7
num_closest_matches = 5
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks=50)   # or pass empty dictionary
flann = cv2.FlannBasedMatcher(index_params,search_params)
error = 0


start = time.time()
for i in range(len(vali_features_shuffled)):   
    print(i)
    vali_descriptors = vali_features_shuffled[i] # Validation Sample

    # stores 5 highest number of keypoint matches
    best_num_matches = np.zeros(num_closest_matches, dtype='uint64')
    
    matches_index = np.zeros(num_closest_matches,dtype='uint64')
    # stores 5 train images with highest number of keypoint matches
    best_image_matches = np.zeros((num_closest_matches, IMG_HEIGHT, IMG_WIDTH, NUM_CHANNELS), dtype='uint8')
    # stores  corresponding matching images 
    
    for j in range(len(train_features_shuffled)):
        # Training sample descriptor
        train_descriptors = train_features_shuffled[j] # Training Sample
        
        # Skip to next training sample if less than 2 local features found, 
        if vali_descriptors is None or train_descriptors is None or \
            len(vali_descriptors) < k_nn or len(train_descriptors) < k_nn:
                continue
                                    ### FOR SIFT ###
        if feature_config == "SIFT": 
            matches = flann.knnMatch(vali_descriptors, train_descriptors, k=k_nn) # Use FLAN for feature matching

            # Create a mask to draw all good matches
            matchesMask = []
            # Store all good matches as per Lowe's Ratio test.
            good = []
            for m,n in matches:
                if m.distance < lowe_ratio *n.distance:
                    good.append(m)
                    matchesMask.append([1,0]) # Match
                else:
                    matchesMask.append([0,0]) # Mismatch
            num_matches = len(good)
            
                                    ### FOR ASIFT ###
        elif feature_config == "ASIFT":
            # Get keypoints for training and validation samples
            kp1 = vali_kp_shuffled[i]
            kp2 = train_kp_shuffled[j]
            
            if len(vali_descriptors) < k_nn or len(train_descriptors) < k_nn: # Skip if less than 2 features found
                continue
            # Use FLANN for feature matching
            raw_matches = matcher.knnMatch(vali_descriptors, trainDescriptors = train_descriptors, k = k_nn) # Use FLANN for feature matching
            
            # Since the above line returns a lot of features, we filter the features.
            p1, p2, kp_pairs = filter_matches(kp1, kp2, raw_matches) 

            if p1.shape[0] < 4 or p2.shape[0] < 4:
                continue

            H, status = cv2.findHomography(p1, p2, cv2.RANSAC, 5.0)
            # Where H is the resulting single-strain matrix.
            # status returns a list of feature points that represent successful matches.
            # ptsA, ptsB are keypoints.
            # The three parameters cv2.RANSAC, ransacReprojThreshold, maxIters are related to RANSAC.
            # ransacReprojThreshold: Maximum reprojection error in the RANSAC algorithm to consider a point as an inlier. 
            # maxIters: The maximum number of RANSAC-based robust method iterations.
            #print('%d / %d  inliers/matched' % (np.sum(status), len(status)))
            
            # Number of good matches
            num_matches = np.sum(status)


        # compares to closest matches and update as necessary
        for k in range(num_closest_matches):
            if num_matches >= best_num_matches[k]:
                best_num_matches = np.insert(best_num_matches, k, num_matches, 0)
                best_num_matches = np.delete(best_num_matches, -1, 0)
                matches_index = np.insert(matches_index, k, j, 0)
                matches_index = np.delete(matches_index, -1, 0)
                #best_image_matches = np.insert(best_image_matches, k, train_image, 0)
                #best_image_matches = np.delete(best_image_matches, -1, 0)
                #circles_and_lines = np.insert(circles_and_lines, k, good_matches, 0)
                #circles_and_lines = np.delete(circles_and_lines, -1, 0)
                break
    
    # Calculate error
    prediction = train_labels_shuffled[matches_index[0]] 
    actual = vali_labels_shuffled[i]
    err = abs(actual[0]-prediction[0]) + abs(actual[1]-prediction[1])
    error +=err
    
    print("MAE",np.round(error/(i+1),2))
    
    print("Prediction:({},{}), Actual:({},{}),ERR{}".
          format(np.round(prediction[0],2),np.round(prediction[1],2),np.round(actual[0],2),np.round(actual[1],2),err))
end = time.time()

MAE = np.round(error/len(vali_features_shuffled),2)
print("Time Taken",end-start)
print("MAE FOR {}:{}".format(feature_config,MAE))

USING FEATURES SIFT
0
MAE 270.9
Prediction:(11.82,-67.22), Actual:(-82.78,109.08),ERR270.89999995
1


KeyboardInterrupt: 

### FEATURE MATCHING: MLP

In [ ]:
if feature_config == "ASIFT" or feature_config == "SIFT":
    print("Cannt use KNN with {}, please use a different descriptor".format(feature_config))
    assert(False)
    
if feature_config == "VLAD":
    input_size = 4096
elif feature_config == "SS ROT" or feature_config == "SS WARP":
    input_size = 256

    
    
model = keras.Sequential(
    [
        layers.Input((input_size,)),
        layers.Dense(300, activation='relu'),
        layers.Dense(300, activation='relu'),
        layers.Dense(2, activation='linear') #regression
    ]
)


model.compile(optimizer='adam', loss='mean_absolute_error')
start=time.time()
history = model.fit(img_features, train_labels, verbose=1,
                                               validation_split = 0.2, epochs=200, batch_size=100)
end=time.time()

print("Time Taken:",end-start)

### FEATURE MATCHING: K NEAREST NEIGHBOURS

In [ ]:
if feature_config == "ASIFT" or feature_config == "SIFT":
    print("Cannt use KNN with {}, please use a different descriptor".format(feature_config))
    assert(False)
start=time.time()
#Shuffle data
idx = np.random.randint(len(img_features), size=len(img_features))
img_shuffled = [img_features[i] for i in idx]
labels_shuffled = [train_labels[i] for i in idx]

# Split into training (80%) and test set (20%)
train_features_shuffled = img_shuffled[:int(len(img_shuffled)*.80)]
train_labels_shuffled = labels_shuffled[:int(len(labels_shuffled)*.80)]

vali_features_shuffled = img_shuffled[int(len(img_shuffled)*.80):]
vali_labels_shuffled = labels_shuffled[int(len(labels_shuffled)*.80):]

# Train
neighbours_model = NearestNeighbors(n_neighbors=1, algorithm='brute', metric='euclidean').fit(train_features_shuffled)
# Predict
distances, indices = neighbours_model.kneighbors(vali_features_shuffled)

test_labels = []
err = 0 
for i in range(len(indices)):
               
    prediction_index = indices[i][0]
    p = train_labels_shuffled[prediction_index]
    actual = vali_labels_shuffled[i]
    
    err += abs(actual[0] - p[0]) + abs(actual[1] - p[1])
    
    print("Predict{}, Actual{}, ERR{}".format(p,actual,err))
    
MAE = err/len(indices)

print("Mean Absolute Error:",MAE)
end=time.time()

print("Time Taken",end-start)

## OUTPUT CSV FILE FOR NETVLAD + KNN

In [ ]:
if feature_config != "VLAD":
    print("Please select NetVLAD as a feature extractor")
    assert(False)
# Train
neighbours_model = NearestNeighbors(n_neighbors=1, algorithm='brute', metric='euclidean').fit(img_features)
# Predict
netvlad_model = NetVLADModel(input_shape=(IMG_HEIGHT, IMG_WIDTH, NUM_CHANNELS)) # Define input layer

netvlad_model.summary()
netvlad_model.load_weights('checkpoint/netvlad_weights.h5')
netvlad_model.build()
test_features = netvlad_model.predict(test_images)
distances, indices = neighbours_model.kneighbors(test_features)

test_labels = []
for i in indices:
    label = train_img_labels[i[0]]
    test_labels.append(label)
    
header = ['id', 'x', 'y']
with open(ROOTPATH + 'predictions_netvlad_knn123.csv', 'w', newline='') as predictions:
    writer = csv.writer(predictions)
    writer.writerow(header)
    for i in range(len(test_img_names)):
        img_name = test_img_names[i]
        x_val, y_val = test_labels[i]
        data = [img_name, str(x_val), str(y_val)]
        writer.writerow(data)
    predictions.close()